In [ ]:
cd ../..

In [3]:
import scipy.io as scio

from pipeline.ingest import loaders

# ---- Sample code to load .mat ----

### The MATLAB ***table*** type

Python's MATLAB format loader (the `scipy` package) cannot process datatype MATLAB's ***table*** 

Thus on the MATLAB side, the ***table*** needs to be converted into ***struct***. See following example:

```
    % Convert matlab "table" into struct
    session.pass.dataTable_struct = struct(session.pass.dataTable)
    session.pass.dataTable_colnames = session.pass.dataTable_struct.varDim.labels
```

### Read `.mat` data

#### read `.mat` from a specified filepath

In [6]:
sess_fp = r'F:\WangLab\sample_data\session.mat'
mat_data = scio.loadmat(sess_fp, struct_as_record = False, squeeze_me = True)

#### Get the `session` variable 

In [7]:
session_mat = mat_data['session']

In [9]:
# show all the field names
session_mat._fieldnames

['rec_id',
 'frameRate',
 'ephysSamplingRate',
 'sweepStartTimes',
 'sweepEndTimes',
 'frametimes',
 'sortedSweepNums',
 'sortedSpikeTimes',
 'sortedSpikeWaveforms',
 'mean_pixel_value',
 'nopass',
 'pass',
 'sortedSweepTimeStart',
 'sortedSweepTimeEnd',
 'mainFolder',
 'videoFolder',
 'min_abs',
 'zaber_order',
 'nopass_id',
 'pass_id']

Two ways to access these fields:
1. the `dot` notation: `session_mat.frameRate`
2. use the `getattr()` method: `getattr(session_mat, 'frameRate')`


In [11]:
session_mat.frameRate

500

In [12]:
getattr(session_mat, 'frameRate')

500

### Example on extracting data from the `pass` and `nopass` task protocols

In [10]:
protocol = 'pass'

In [13]:
data = getattr(session_mat, 'pass')

In [15]:
data._fieldnames

['start_frame',
 'end_frame',
 'trial_num',
 'trial_start',
 'trial_end',
 'trial_start_auto',
 'trial_end_auto',
 'zaber_position',
 'direction',
 'wall_distance',
 'trial_start_time',
 'trial_end_time',
 'trial_spike_times',
 'sorted_spike_trials',
 'dataTable',
 'dataTable_struct',
 'dataTable_colnames']

#### Read the MATLAB ***table***

`dataTable_struct` and `dataTable_colnames` are the two additional fields added in MATLAB to allow for extracting MATLAB's ***table*** in python.

To read this, I've built a helper function named `load_table_from_struct` from `pipeline.ingest.loaders` to read this `dataTable_struct` into a pandas DataFrame

In [17]:
data.dataTable_struct

In [18]:
data.dataTable_colnames

array(['trialNum', 'spikeTimes', 'firingRate', 'wallDistanceMM',
       'wallDistancePixel', 'wallDirection', 'startTime', 'endTime',
       'passTime'], dtype=object)

In [19]:
data_df = loaders.load_table_from_struct(data.dataTable_struct, data.dataTable_colnames)

In [20]:
data_df

,trialNum,spikeTimes,firingRate,wallDistanceMM,wallDistancePixel,wallDirection,startTime,endTime,passTime
0,1,"[1581.36185, 1581.3888, 1581.3994, 1581.42715,...",24.950495,3,330.847314,forward,1581.348,1586.398,5.05
1,2,"[1595.84625, 1595.9269, 1596.06765, 1596.07155...",24.732620,1,380.711879,forward,1593.798,1601.278,7.48
2,3,"[1611.12095, 1611.1341499999999, 1611.1632, 16...",9.342105,4,299.989597,forward,1608.638,1616.238,7.60
3,4,"[1627.6302, 1627.69055, 1627.7516, 1627.81095,...",3.740157,10,112.613203,forward,1625.518,1630.598,5.08
4,5,"[1642.34215, 1642.4304, 1642.47505, 1642.78625...",1.394422,12,51.074472,forward,1639.998,1645.018,5.02
5,6,"[1656.33865, 1656.3835, 1656.5079, 1656.52, 16...",6.809339,7,206.312515,forward,1654.498,1659.638,5.14
6,7,"[1669.47645, 1669.5659, 1669.5758, 1669.5888, ...",5.098039,8,175.189775,forward,1669.118,1674.218,5.10
7,8,"[1685.96395, 1686.0182, 1686.0326, 1686.0805, ...",10.000000,5,268.801633,forward,1683.798,1688.898,5.10
8,9,"[1697.8296, 1698.5724, 1698.59495, 1698.6114, ...",6.368564,6,237.688479,forward,1696.198,1703.578,7.38
9,10,"[1714.9099, 1715.1138999999998, 1715.18415, 17...",2.354571,9,143.988668,forward,1710.898,1718.118,7.22


#### Now you can access and get any relevant data out by interacting with the DataFrame